In [5]:
Import-Module dbatools, psopenai, ./dbatools.ai.psd1

In [ ]:
Request-ChatCompletion -Message "Who are you?" | Select-Object -ExpandProperty Answer

In [ ]:
Request-ChatGPT -Message "Hi, please tell me your name."

In [ ]:
# Just some quick cleanup if needed
Get-ModuleAssistant -Name 'finetuna copilot' | Remove-ModuleAssistant

In [ ]:
# Create new assistant and set the default module to use
New-ModuleAssistant -Module finetuna

In [ ]:
# uses alias for Invoke-HelpChat
askhelp how do I get started?

In [ ]:
Set-ModuleAssistant -Module dbatools

In [ ]:
askhelp how do i encrypt all of my databases?